In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.CDSSM.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 8100.58it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:05<00:00, 3725.90it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 3256133.23it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9243.16it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4432.30it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 1013100.54it/s]
Processing text_left with chain_transform of NgramLetter => WordHashing: 100%|██████████| 2118/2118 [00:02<00:00, 806.09it/s]
Processing text_right with chain_tra

In [4]:
preprocessor.context

{'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7efd619b6f98>,
 'vocab_size': 9645,
 'embedding_input_dim': 9645}

In [5]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
)

In [6]:
padding_callback = mz.models.CDSSM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    sort=False,
    resample=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    callback=padding_callback
)

In [7]:
model = mz.models.CDSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['vocab_size']
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['conv_activation_func'] = 'tanh'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

CDSSM(
  (net_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(9645, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (net_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(9645, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): T

In [8]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [9]:
trainer.run()

[Iter-102 Loss-0.860]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4493 - normalized_discounted_cumulative_gain@5(0.0): 0.5235 - mean_average_precision(0.0): 0.4821



[Iter-204 Loss-0.658]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4713 - normalized_discounted_cumulative_gain@5(0.0): 0.5352 - mean_average_precision(0.0): 0.5028



[Iter-306 Loss-0.545]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4671 - normalized_discounted_cumulative_gain@5(0.0): 0.5394 - mean_average_precision(0.0): 0.4947



[Iter-408 Loss-0.420]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4646 - normalized_discounted_cumulative_gain@5(0.0): 0.5377 - mean_average_precision(0.0): 0.4995



[Iter-510 Loss-0.362]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4769 - normalized_discounted_cumulative_gain@5(0.0): 0.5535 - mean_average_precision(0.0): 0.5109



[Iter-612 Loss-0.306]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4785 - normalized_discounted_cumulative_gain@5(0.0): 0.5487 - mean_average_precision(0.0): 0.5108



[Iter-714 Loss-0.237]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5008 - normalized_discounted_cumulative_gain@5(0.0): 0.5632 - mean_average_precision(0.0): 0.5312



[Iter-816 Loss-0.208]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4856 - normalized_discounted_cumulative_gain@5(0.0): 0.5526 - mean_average_precision(0.0): 0.5164



[Iter-918 Loss-0.199]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4862 - normalized_discounted_cumulative_gain@5(0.0): 0.5457 - mean_average_precision(0.0): 0.5118



[Iter-1020 Loss-0.178]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4813 - normalized_discounted_cumulative_gain@5(0.0): 0.5523 - mean_average_precision(0.0): 0.5128

Cost time: 4880.596559524536s
